In [1]:
import pandas as pd
import os
import numpy as np
import requests
import datetime
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import datetime
import time
import json
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MultipleLocator
import openmeteo_requests
import requests_cache
from retry_requests import retry
import hsfs
from pathlib import Path
from dotenv import load_dotenv
import hopsworks
import sys

root_dir = Path().resolve().parent
sys.path.append(str(root_dir))

from format_data import format_weather_data, format_price_data
from get_electricity_prices import get_data, get_todays_data
from get_weather_data import get_historical_weather, get_weather_forecast



In [2]:
load_dotenv()
hopsworks_api = os.getenv("HOPSWORKS_API_KEY")

os.environ["HOPSWORKS_API_KEY"] = hopsworks_api

project = hopsworks.login()
fs = project.get_feature_store() 
print(f"Connected to project: {project.name}")

2024-12-26 17:01:37,908 INFO: Initializing external client
2024-12-26 17:01:37,910 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-26 17:01:39,629 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164446
Connected to project: oskaralf


In [3]:
forecast = get_weather_forecast("Luleå", "X", "X", 65.5841, 22.1547)
print(forecast)

Coordinates 65.58380889892578°N 22.155181884765625°E
Elevation 15.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
              time_start  temperature_2m  precipitation  snow_depth  \
0    2024-12-26T00:00:00           1.574            0.0        0.17   
1    2024-12-26T01:00:00           1.524            0.0        0.17   
2    2024-12-26T02:00:00           1.774            0.0        0.17   
3    2024-12-26T03:00:00           2.074            0.0        0.17   
4    2024-12-26T04:00:00           2.024            0.0        0.17   
..                   ...             ...            ...         ...   
163  2025-01-01T19:00:00          -5.700            0.1        0.13   
164  2025-01-01T20:00:00          -4.950            0.1        0.13   
165  2025-01-01T21:00:00          -4.350            0.1        0.13   
166  2025-01-01T22:00:00          -4.000            0.1        0.13   
167  2025-01-01T23:00:00          -3.750            0.1        0.13   

     pressure_msl  cl

In [4]:
price_fg = fs.get_feature_group(
    name='electricity_price_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)

In [5]:
weather_fg.insert(forecast)

2024-12-26 17:01:47,300 INFO: 	5 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393145


Uploading Dataframe: 100.00% |██████████| Rows 168/168 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: weather_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/weather_data_1_offline_fg_materialization/executions


(Job('weather_data_1_offline_fg_materialization', 'SPARK'),
 {
   "success": false,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "temperature_2m",
           "min_value": -100.0,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 695302
         }
       },
       "result": {
         "observed_value": -10.550000190734863,
         "element_count": 168,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-12-26T04:01:47.000300Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_

In [6]:
price = get_todays_data("SE4")
price['time_start'] = price['time_start'].str.replace(r'\+\d{2}:\d{2}$', '', regex=True)
print(price)
print(price.dtypes)

         date           time_start    price
0  2024-12-26  2024-12-26T00:00:00  0.02900
1  2024-12-26  2024-12-26T01:00:00  0.02230
2  2024-12-26  2024-12-26T02:00:00  0.01490
3  2024-12-26  2024-12-26T03:00:00 -0.00127
4  2024-12-26  2024-12-26T04:00:00 -0.00381
5  2024-12-26  2024-12-26T05:00:00 -0.00393
6  2024-12-26  2024-12-26T06:00:00 -0.00266
7  2024-12-26  2024-12-26T07:00:00 -0.00439
8  2024-12-26  2024-12-26T08:00:00 -0.00439
9  2024-12-26  2024-12-26T09:00:00  0.00739
10 2024-12-26  2024-12-26T10:00:00  0.02207
11 2024-12-26  2024-12-26T11:00:00  0.02611
12 2024-12-26  2024-12-26T12:00:00  0.05765
13 2024-12-26  2024-12-26T13:00:00  0.26213
14 2024-12-26  2024-12-26T14:00:00  0.31193
15 2024-12-26  2024-12-26T15:00:00  0.33157
16 2024-12-26  2024-12-26T16:00:00  0.41325
17 2024-12-26  2024-12-26T17:00:00  0.37350
18 2024-12-26  2024-12-26T18:00:00  0.37166
19 2024-12-26  2024-12-26T19:00:00  0.36426
20 2024-12-26  2024-12-26T20:00:00  0.33064
21 2024-12-26  2024-12-26T21:00:

In [7]:
price_fg.insert(price)

2024-12-26 17:02:06,500 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393146


Uploading Dataframe: 100.00% |██████████| Rows 24/24 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: electricity_price_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/electricity_price_data_1_offline_fg_materialization/executions


(Job('electricity_price_data_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "price",
           "min_value": -0.1,
           "max_value": 5000,
           "strict_min": true
         },
         "meta": {
           "expectationId": 695305
         }
       },
       "result": {
         "observed_value": -0.00439,
         "element_count": 24,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-12-26T04:02:06.000500Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "suc